In [ ]:
import os
import os.path
import numpy as np
from IPython.display import display
import ipywidgets as widgets

def H1(text):
    return widgets.HTML(f"<h1>{text}</h1>")
    
def H2(text):
    return widgets.HTML(f"<h2>{text}</h2>")

def evaluate_all(data_dir):
    # Brechkraft
    def D(g, b):
        return 1 / g + 1 / b
    # Brechkraft System
    def Ds(f1, f2, d):
        return 1 / f1 + 1 / f2 - d / (f1 * f2)
    # Brennweite
    def f(g, b):
        return 1 / D(g, b)
    # Brennweite System
    def fs(f1, f2, d):
        return 1 / Ds(f1, f2, d)
    
    display(H1("Teil 1: Sammellinsen"))
    
    display(H2("Linse 1"))
    data = np.load(os.path.join(data_dir, "DataV4041Part1_Lens1.npy"))
    objectDistance1 = data[0] * 1e-2 # m
    imageDistance1 = data[1] * 1e-2 # m

    focusDistance1 = f(objectDistance1, imageDistance1)
    refractivity1 = 1 / focusDistance1

    print("Brennweite [m]:\n", np.round(focusDistance1, 3))
    print("Mittelwert Brennweite = {0:.3f} m".format(np.mean(focusDistance1)))
    print("Brechkraft [dpt]:\n", np.round(refractivity1, 3))
    print("Mittelwert Brechkraft = {0:.3f} dpt".format(np.mean(refractivity1)))

    display(H2("Linse 2"))
    data = np.load(os.path.join(data_dir, "DataV4041Part1_Lens2.npy"))
    objectDistance2 = data[0] * 1e-2 # m
    imageDistance2 = data[1] * 1e-2 # m

    focusDistance2 = f(objectDistance2, imageDistance2)
    refractivity2 = 1 / focusDistance2

    print("Brennweite [m]:\n", np.round(focusDistance2, 3))
    print("Mittelwert Brennweite = {0:.3f} m".format(np.mean(focusDistance2)))
    print("Brechkraft [dpt]:\n", np.round(refractivity2, 3))
    print("Mittelwert Brechkraft = {0:.3f} dpt".format(np.mean(refractivity2)))
    
    display(H1("Teil 2: Linsensystem"))
    data = np.load(os.path.join(data_dir, "DataV4041Part2.npy"))
    objectDistance3 = data[0] * 1e-2 # m
    imageDistance3 = data[1] * 1e-2 # m
    lenseDistance3 = 0.028 # m

    focusDistance3 = f(objectDistance3, imageDistance3)
    refractivity3 = 1 / focusDistance3

    focusDistance3_system = fs(np.mean(focusDistance1), np.mean(focusDistance2), lenseDistance3)
    refractivity3_system = 1 / focusDistance3_system

    focusDistance3_approx_system = fs(np.mean(focusDistance1), np.mean(focusDistance2), 0)
    refractivity3_approx_system = 1 / focusDistance3_approx_system

    print("Brennweite [m]:\n", np.round(focusDistance3, 3))
    print("Brechkraft [dpt]:\n", np.round(refractivity3, 3))
    print("_____________________")
    print("Linsensystemformel:")
    print("Exaktere Werte:")
    print("Brennweite = {0:.3f} m".format(focusDistance3_system))
    print("Brechkraft = {0:.3f} dpt".format(refractivity3_system))
    print()
    print("Mit Näherungsformel:")
    print("Brennweite = {0:.3f} m".format(focusDistance3_approx_system))
    print("Brechkraft = {0:.3f} dpt".format(refractivity3_approx_system))
    print()
    print("Relative Differenzen:")
    print("Brennweite (exp - approx) : {0} %".format(round((focusDistance3 - focusDistance3_approx_system) / focusDistance3 * 100, 1)))
    print("Brechkraft (exp - approx) : {0} %".format(round((refractivity3 - refractivity3_approx_system) / refractivity3 * 100, 1)))
    print("Brennweite (exp - exact) : {0} %".format(round((focusDistance3 - focusDistance3_system) / focusDistance3 * 100, 1)))
    print("Brechkraft (exp - exact) : {0} %".format(round((refractivity3 - refractivity3_system) / refractivity3 * 100, 1)))

    display(H1("Teil 3: Zerstreulinse"))
    data = np.load(os.path.join(data_dir, "DataV4041Part3.npy"))
    objectDistance4 = data[0] * 1e-2 # m
    imageDistance4 = data[1] * 1e-2 # m
    lenseDistance4 = 0.03 # m

    focusDistance4 = f(objectDistance4, imageDistance4)
    refractivity4 = 1 / focusDistance4
    refractivityLens3 = refractivity4 - np.mean(refractivity1)
    focusDistanceLens3 = 1 / refractivityLens3

    print("Brennweite System = {0:.3f} m".format(focusDistance4))
    print("Brechkraft Sytsem = {0:.3f} dpt".format(refractivity4))
    print("Brennweite Linse3 = {0:.3f} m".format(focusDistanceLens3))
    print("Brechkraft Linse3 = {0:.3f} dpt".format(refractivityLens3))
    

# Interactive student selection
with open("students.txt", "r") as f:
    students = [line.strip("\n") for line in f if not line.startswith("#")]
    
tb = widgets.ToggleButtons(
    options=students,
    description='Name:',
)
display(tb)

output = widgets.Output()
display(output)

@output.capture(clear_output=True)
def on_student_change(change):
    name = change["new"]
    display(H1(f"Auswertung für: {name}"))

    name_escaped = name.replace(" ", "_")
    data_dir = os.path.join("V4041", name_escaped)
    
    evaluate_all(data_dir)

tb.observe(on_student_change, names="value")
on_student_change({"new": students[0]})